# User chunk

| Cột              | Giữ / Loại bỏ       | Lý do                                                                                                               |
| ---------------- | ------------------- | ------------------------------------------------------------------------------------------------------------------- |
| `customer_id`, `user_id`    | ✅ Giữ| các biến định danh, không mang ý nghĩa thống kê trong phân tích dữ liệu. Nhưng  có thể giữ lại để liên kết với purchase chunk và item chunk|
| `gender`         | ✅ Giữ               | Có thể ảnh hưởng đến hành vi mua hàng                                                                               |
| `location`       | ✅ Giữ               | Thể hiện mã khu vực – hữu ích trong phân tích địa lý                                                                |
| `province`       | ✅ Giữ               | Dùng phân tích xu hướng theo địa phương                                                                             |
| `region`         | ✅ Giữ               | Cung cấp thông tin phân cấp cao hơn, có thể hữu ích trong mô hình                                                   |
| `membership`     | ✅ Giữ               | Ảnh hưởng lớn đến hành vi và giá trị khách hàng                                                                     |
| `sync_status_id` | ❌ Loại bỏ           | Gần như toàn bộ giá trị Null, phần còn lại đều bằng 2 → không có giá trị phân biệt                                  |
| `location_name`  | ❌ Loại bỏ           | Dữ liệu quá chi tiết, rời rạc và không khái quát được                                                               |
| `install_app`    | ✅ Giữ               | Kênh cài đặt có thể ảnh hưởng đến tần suất mua hàng                                                                 |
| `district`       | ⚙️ Giữ có điều kiện | Có thể giữ nếu cần phân tích chi tiết theo khu vực; nếu không, loại bỏ do trùng lặp thông tin địa lý với `province` |
| `is_deleted`     | ❌ Loại bỏ           | Toàn bộ là `False`, không mang thông tin                                                                            |


In [3]:
import os
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [4]:
def read_parquet_by_type(train_path: str):
    # Lấy tất cả các file parquet trong thư mục
    files = [os.path.join(train_path, f) for f in os.listdir(train_path) if f.endswith('.parquet')]
    
    # Phân loại các file theo loại tên
    user_chunk_files = [file for file in files if 'user_chunk' in file]
    purchase_history_chunk_files = [file for file in files if 'purchase_history_daily_chunk' in file]
    item_chunk_files = [file for file in files if 'item_chunk' in file]
    
    # Đọc các file riêng biệt thành DataFrame
    user_chunk_df = pl.concat([pl.read_parquet(file) for file in user_chunk_files]) if user_chunk_files else None
    purchase_history_chunk_df = pl.concat([pl.read_parquet(file) for file in purchase_history_chunk_files]) if purchase_history_chunk_files else None
    item_chunk_df = pl.concat([pl.read_parquet(file) for file in item_chunk_files]) if item_chunk_files else None
    
    # Trả về một dictionary chứa các DataFrame
    return {
        "user_chunk": user_chunk_df,
        "purchase_history_chunk": purchase_history_chunk_df,
        "item_chunk": item_chunk_df
    }

In [5]:
train_path = 'E:\KHMT2023_CS_UIT/05_C_Python_For_ML/recommendation_dataset'
dataframes = read_parquet_by_type(train_path)

df_user = dataframes["user_chunk"]
df_purchase = dataframes["purchase_history_chunk"]
df_item = dataframes["item_chunk"]

In [6]:
cols_to_keep_user = [
    'customer_id', 'user_id', 'gender', 'location', 'province',
    'region', 'membership', 'install_app'
]

df_user = df_user.select(cols_to_keep_user)

print("User Chunk DataFrame:")
df_user.head()

User Chunk DataFrame:


customer_id,user_id,gender,location,province,region,membership,install_app
i32,str,str,i32,str,str,str,str
14732,"""e1e48206652bf8c279ff0206c69a80…","""Nam""",155,"""Hồ Chí Minh""","""Đông Nam Bộ""","""Standard""","""In-Store"""
15126,"""77891759204bd27e69fb11a7b92889…","""Nữ""",300,"""Hồ Chí Minh""","""Đông Nam Bộ""","""Standard""","""In-Store"""
29718,"""b8041b584a0bb6655361727a0a6108…","""Nữ""",157,"""Bến Tre""","""Đồng bằng sông Cửu Long""","""Standard""","""In-Store"""
30077,"""c52ceaca44a83ec41a219cfaff0e05…","""Nữ""",53,"""Hồ Chí Minh""","""Đông Nam Bộ""","""Standard""","""In-Store"""
30085,"""74ac5764f5d67d341aacf238d7762f…","""Nữ""",660,"""Hồ Chí Minh""","""Đông Nam Bộ""","""Standard""","""In-Store"""


# Purchase chunk

| Cột            | Giữ / Loại bỏ | Lý do                                                                                                                                             |
| -------------- | ------------- | ------------------------------------------------------------------------------------------------------------------------------------------------- |
| `timestamp`    | ✅ Giữ         | Biểu diễn thời gian giao dịch, có thể phân tích xu hướng theo thời gian; cần chuyển từ UNIX timestamp sang dạng datetime để thuận tiện phân tích. |
| `user_id`      | ✅ Giữ         | Biến định danh người dùng, cần để liên kết với bảng `User Chunk`; không có ý nghĩa thống kê độc lập nhưng quan trọng cho join dữ liệu.            |
| `item_id`      | ✅ Giữ         | Dùng để xác định sản phẩm và phân tích tần suất, mức độ phổ biến của mặt hàng.                                                                    |
| `event_type`   | ❌ Loại bỏ     | Toàn bộ dữ liệu chỉ có giá trị “Purchase” → không mang thông tin phân biệt.                                                                       |
| `event_value`  | ❌ Loại bỏ     | Tương quan hoàn hảo với `quantity` (r = 1.0) → gây multicollinearity, nên giữ `quantity`.                                                         |
| `price`        | ✅ Giữ         | Biến quan trọng để phân tích doanh thu, hành vi giá và khuyến mãi; cần xử lý outlier (trimming hoặc winsorizing).                                 |
| `quantity`     | ✅ Giữ         | Thể hiện số lượng mua; cần xử lý outlier (giá trị quá lớn có thể là đơn hàng sỉ hoặc lỗi).                                                        |
| `date_key`     | ❌ Loại bỏ     | Tương quan gần tuyệt đối với `timestamp` (r ≈ 0.998) → dư thừa thông tin thời gian.                                                               |
| `customer_id`  | ✅ Giữ         | Biến định danh khách hàng, cần để liên kết với bảng `User Chunk`.                                                                                 |
| `created_date` | ✅ Giữ         | Thời điểm tạo đơn hàng, có thể khác thời điểm mua (timestamp); hữu ích trong phân tích hành vi và vận hành.                                       |
| `payment`      | ✅ Giữ         | Thông tin phương thức thanh toán (cash, transfer, v.v.) → quan trọng trong phân tích kênh thanh toán và hành vi người dùng.                       |
| `location`     | ✅ Giữ         | Cho phép phân tích xu hướng mua hàng theo khu vực; không có outlier.                                                                              |
| `discount`     | ✅ Giữ         | Thể hiện mức chiết khấu; cần chuẩn hóa hoặc winsorize do phân bố lệch và nhiều outlier.                                                           |
| `channel`      | ✅ Giữ         | Phân biệt kênh bán hàng (In-Store, Chat, CRM, v.v.), có mối liên hệ mạnh với `payment` → hữu ích trong phân tích hành vi.                         |
| `is_deleted`   | ❌ Loại bỏ     | Toàn bộ giá trị là `False` → không mang thông tin phân biệt.                                                                                      |


In [8]:
# Các cột cần giữ lại 
cols_to_keep_purchase = [
    'customer_id', 'user_id', 'item_id', 'created_date', 'price', 
    'timestamp', 'quantity', 'location', 'discount', 'channel', 'payment'
]

# Chọn các cột cần giữ
df_purchase = df_purchase.select(cols_to_keep_purchase)

print("Purchase Chunk DataFrame:")
df_purchase.head()


Purchase Chunk DataFrame:


customer_id,user_id,item_id,created_date,price,timestamp,quantity,location,discount,channel,payment
i32,str,str,datetime[μs],"decimal[38,4]",i64,i32,i32,"decimal[38,4]",str,str
5254214,"""ca12702ddf55acaa9fb767e10faaa6…","""7115000000004""",2024-12-24 18:17:01.027,49000.0000,1735064221,1,656,0.0000,"""In-Store""","""VietQR"""
7573232,"""54314ce86269f4849f9a4c3fc85d94…","""0029130000030""",2024-12-24 19:28:01.870,69000.0000,1735068481,1,143,0.0000,"""In-Store""","""Tiền mặt"""
8187418,"""4799de4022b58689b2b9fb37122479…","""3496000000053""",2024-12-24 19:50:43.760,75000.0000,1735069843,2,213,0.0000,"""In-Store""","""MoMo"""
8187418,"""4799de4022b58689b2b9fb37122479…","""2700000000002""",2024-12-24 19:49:39.223,58500.0000,1735069779,2,213,13000.0000,"""In-Store""","""MoMo"""
6931560,"""3b2423f730815273dfd6f6220b0c5c…","""0029110000036""",2024-12-28 09:49:33.780,89000.0000,1735379373,1,590,10000.0000,"""Android""","""MoMo"""


# Item chunk

| Cột                  | Giữ / Loại bỏ       | Lý do                                                                                                                                                    |
| -------------------- | ------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------- |
| `item_id`            | ❌ Loại bỏ           | Mỗi giá trị đều duy nhất, chỉ mang tính định danh, không cung cấp thông tin thống kê; giữ lại chỉ khi cần join với bảng khác (`purchase`).               |
| `is_deleted`         | ❌ Loại bỏ           | Tất cả giá trị đều là `False` → không mang thông tin phân biệt.                                                                                          |
| `created_date`       | ✅ Giữ               | Cho biết thời điểm tạo sản phẩm, có thể dùng phân tích vòng đời sản phẩm, xác định hàng cũ / hàng mới.                                                   |
| `creation_timestamp` | ✅ Giữ               | Biểu diễn thời gian tạo ở dạng timestamp, có thể dùng phân tích xu hướng sản phẩm theo thời gian; nên chuyển sang dạng datetime để trực quan hóa dễ hơn. |
| `price`              | ✅ Giữ               | Biến chính trong phân tích lợi nhuận và định giá sản phẩm; có nhiều outlier nên cần trimming hoặc winsorizing trước khi phân tích.                       |
| `gp` (gross profit)  | ✅ Giữ               | Thể hiện lợi nhuận gộp, tương quan cao với `price` (r = 0.8867); có ý nghĩa kinh doanh mạnh, dùng để phân tích hiệu quả từng nhóm sản phẩm.              |
| `weight`             | ⚙️ Giữ có điều kiện | Có nhiều giá trị 0 hoặc “Không xác định”; có thể giữ lại nếu muốn phân tích theo khối lượng hoặc vận chuyển, nhưng cần chuẩn hóa đơn vị đo.              |
| `category_l1`        | ✅ Giữ               | Danh mục cấp 1 – hữu ích cho phân tích xu hướng ngành hàng lớn, nhưng cần xử lý mất cân đối khi huấn luyện mô hình.                                      |
| `category_l2`        | ✅ Giữ               | Danh mục cấp 2 – thể hiện rõ sự khác biệt trong khả năng tiêu thụ sản phẩm (`sale_status` khác nhau đáng kể giữa các nhóm).                              |
| `category_l3`        | ✅ Giữ               | Danh mục cấp 3 – phản ánh phân khúc chi tiết của sản phẩm; cần gộp nhóm các nhãn hiếm (rare categories) để giảm nhiễu khi mã hóa.                        |
| `brand`              | ⚙️ Giữ có điều kiện | Khoảng 70% giá trị bị thiếu hoặc “Không xác định”; có thể giữ nếu được làm sạch và gộp nhóm “Unknown”, nếu không thì nên loại bỏ do thiếu dữ liệu.       |
| `manufacturer`       | ❌ Loại bỏ           | Hơn 94% giá trị “Không xác định” → không mang thông tin hữu ích.                                                                                         |
| `gender_target`      | ⚙️ Giữ có điều kiện | 66% “Không xác định”, phần còn lại có thể hữu ích trong phân tích hành vi theo giới tính nếu được làm sạch.                                            |
| `age_group`          | ✅ Giữ               | Cung cấp thông tin độ tuổi mục tiêu (dù mất cân đối nhưng vẫn hữu ích để mô tả sản phẩm).                                                                |
| `sale_status`        | ✅ Giữ               | Phản ánh tình trạng bán (bán được / không bán được); có mối liên hệ rõ với `category_l2`, hữu ích cho phân tích sức tiêu thụ.                            |
| `p_id`               | ✅ Giữ               | Liên quan đến thế hệ mã hàng; giúp nhận diện vòng đời sản phẩm, đặc biệt trong phân tích hàng lỗi thời.                                                  |


In [9]:
# Các cột cần giữ lại
cols_to_keep_item = [
    'created_date', 'creation_timestamp', 'price', 'gp',
    'weight', 'category_l1', 'category_l2', 'category_l3',
    'brand', 'gender_target', 'age_group', 'sale_status', 'p_id'
]

# Chọn các cột cần giữ
df_item = df_item.select(cols_to_keep_item)

print("Item Chunk DataFrame:")
df_item.head()


Item Chunk DataFrame:


created_date,creation_timestamp,price,gp,weight,category_l1,category_l2,category_l3,brand,gender_target,age_group,sale_status,p_id
datetime[μs],i64,"decimal[38,4]","decimal[38,4]",f32,str,str,str,str,str,str,i32,i32
2012-04-04 09:25:44.240,1333531544,99000.0000,36828.0000,null,"""Babycare""","""Bình sữa, phụ kiện""","""Núm ty""","""Dr.Brown's""","""Không xác định""","""Không xác định""",0,17065
2017-08-18 08:50:50.713,1503046250,69000.0000,0.0000,null,"""Thời trang""","""Cơ cấu hàng cũ""","""Thời trang bé trai, bé gái cũ""","""Con Cưng""","""Bé Gái""","""Từ 3Y""",0,72370
2013-01-18 09:33:04.260,1358501584,45000.0000,14490.0000,null,"""Đồ chơi & Sách""","""0-1Y""","""Gặm nướu""","""Thương hiệu khác""","""Không xác định""","""Không xác định""",0,31154
2014-05-14 10:07:19.603,1400062039,401000.0000,59749.0000,null,"""Tã""","""Merries""","""Merries""","""Merries Nhật""","""Không xác định""","""Không xác định""",0,46123
2014-05-14 10:07:20.370,1400062040,401000.0000,65764.0000,null,"""Tã""","""Merries""","""Merries""","""Merries Nhật""","""Không xác định""","""Không xác định""",0,46127
